# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
import numpy as np
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm

# to help generate formatted output representations
from IPython.core.display import display, HTML
import markdown2 as md

def html(markdown_string):
    display(HTML(md.markdown(markdown_string)))

## Helper Function(s)

In [2]:
def _relevantGameInfo(game):
    link = game[:game.find('\n')]
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])
    slots, players, correct = [], [], None
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        if (line[2].lower().count('lynched') > 0 and
            line[2].lower().count('day 1') > 0):
            correct = slots[-1]
    return slots, players, correct, number

## VotecounterTest

In [5]:
#parameters
archive = '../data/archive.txt'
votecounter = VoteCounter
start_index = 0
end_index = 0

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  
    
# process votes in each game's posts until a lynch found
# then store information about votecounter's performance
results, success, t0, total = {}, 0, time.time(), 0
end_index = end_index if end_index else len(games)  
for game_index, game in enumerate(games[start_index:end_index]):
    print()
    print('\n'.join(game.split('\n')[:2]))
    slots, players, correct, number = _relevantGameInfo(game)
    #print(slots, players, correct, number)
    votecount = VoteCount(slots, meta={'correct': correct})
    votecounter = VoteCounter.VoteExtracter(players=players)

    # collect gameposts associated with game number
    with open('../data/posts/{}.jsonl'.format(number)) as f:
        gameposts = [dict(t) for t in {tuple(d.items()) for d in [json.loads(l) for l in f]}]
        gameposts = sorted(gameposts, key=lambda x: (int(x['thread']), int(x['number'])))
        
    tgame = time.time()
    for post in gameposts:
        # done if voters have made a choice already
        if votecount.choice:
            break

        # ignore posts not made by players
        if players.count(post['user']) == 0:
            continue

        # update votecount for each vote found by votecounter
        for voted in votecounter.fromPost(post):
            votecount.update(post['user'], voted, post['number'])
            if votecount.choice:
                success += votecount.choice == correct
                break
    results[number] = votecount
    total += 1
    print(game_index + start_index, number, success, total, votecount.choice == correct, time.time()-tgame)
    
print(success/float(end_index-start_index), time.time()-t0)


https://forum.mafiascum.net/viewtopic.php?f=53&t=15787
Game 1091: Mafia Mania
0 15787 1 1 True 0.17148447036743164

https://forum.mafiascum.net/viewtopic.php?f=53&t=15783
Game 1094: Mariposa Peak Mafia
1 15783 2 2 True 0.13050389289855957

https://forum.mafiascum.net/viewtopic.php?f=53&t=15828
Game 1098: The Mafia Experiment!
2 15828 3 3 True 0.15499472618103027

https://forum.mafiascum.net/viewtopic.php?f=53&t=15862
Game 1101: Suspiciously Normal Mafia
3 15862 4 4 True 0.05449700355529785

https://forum.mafiascum.net/viewtopic.php?f=53&t=15883
Game 1102: Rivertown Mafia
4 15883 5 5 True 0.2805194854736328

https://forum.mafiascum.net/viewtopic.php?f=53&t=15934
Game 1105: A Mafia Invasion!
5 15934 6 6 True 0.12101888656616211

https://forum.mafiascum.net/viewtopic.php?f=53&t=15982
Game 1107: Just a Game
6 15982 7 7 True 0.05950021743774414

https://forum.mafiascum.net/viewtopic.php?f=53&t=16073
Game 1112: Mundania
7 16073 8 8 True 0.04950141906738281

https://forum.mafiascum.net/viewt

## Analyze Results

In [21]:
# parameters
game_index = 222
postnumber = 432
postnumber = str(postnumber)

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# generate votecounter w/ specified players
slots, players, correct, number = _relevantGameInfo(games[game_index])
html('# {}'.format(number))
print(games[game_index])
print()
print('Choice:', results[number].choice)
print('Correct:', results[number].meta['correct'])
votecount = VoteCount(slots, meta={'correct': correct})
votecounter = VoteCounter.VoteExtracter(players=players)

# collect gameposts associated with game number
with open('../data/posts/{}.jsonl'.format(number)) as f:
    gameposts = [dict(t) for t in {tuple(d.items()) for d in [json.loads(l) for l in f]}]
    gameposts = sorted(gameposts, key=lambda x: (int(x['thread']), int(x['number'])))
    
# find and display selected post along with votecounter output for it
html('# Post {}'.format(postnumber))
post = next(item for item in gameposts if item["number"] == postnumber)
print('Extracted Votes:', list(votecounter.fromPost(post)))
print()
print(post)
print()
display(HTML(post['content']))
    
# display votecount up to indicated post number
html('# Current Votecount (Up to {})'.format(postnumber))
for post in gameposts[:int(postnumber)]:
    # done if voters have made a choice already
    if votecount.choice:
        break

    # ignore posts not made by players
    if players.count(post['user']) == 0:
        continue

    # update votecount for each vote found by votecounter
    for voted in votecounter.fromPost(post):
        votecount.update(post['user'], voted, post['number'])
        if votecount.choice:
            success += votecount.choice == correct
            break
            
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

# display final votecount
html('# Final Votecount')
final_votecount = results[number].todict()
for each in final_votecount:
    if final_votecount[each]:
        print(each, '-', len(final_votecount[each]))
        for voter in final_votecount[each]:
            print(voter)
        print()
        
# display final votelog
html('# Vote Log')
votelog = results[number].votelog.copy()
for index, each in enumerate(votelog):
    each = each.split()
    each[-1] = '[{}]({})'.format(each[-1], next(item for item in gameposts if item["number"] == each[-1])['pagelink'])
    votelog[index] = ' '.join(each)
html('  \n'.join(reversed(votelog)))

https://forum.mafiascum.net/viewtopic.php?f=53&t=64784
Mini 1752: Back to December
Moderator: Equinox
Current Update: Town Win

Players
Aj The Epic, mafia goon, lynched day 2
Alchemist21, town even-night tracker, survived
curiouskarmadog, town vanilla, survived
Elyse, town vanilla, survived
LlamaFluff, town vanilla, survived
Marquis, town vanilla, killed night 0
Xtoxm replaced MattP, town vanilla, survived
OceanWind, town vanilla, survived
petroleumjelly, town vanilla, killed night 1
Raskolnikov, town bodyguard, survived
Supreme Overlord, town vanilla, survived
TKoE, mafia commuter, lynched day 1

Choice: ['Aj The Epic']
Correct: ['TKoE']


Extracted Votes: []

{'forum': '53', 'thread': '64784', 'pagelink': 'https://forum.mafiascum.net/viewtopic.php?f=53&t=64784&start=425', 'number': '432', 'timestamp': 'Sun Jan 17, 2016 1:30 am', 'user': 'Equinox', 'content': '<span style="color: #BF0000"><span class="nocolorsig"><fieldset style="border:3px outset grey; padding:5px 10px"><legend style="text-transform:uppercase; margin:0px 0.6em; padding:0em 0.33em">Vote Count 1.17</legend>TKoE (6) - Raskolnikov, Elyse, Xtoxm, Alchemist21, Supreme Overlord, curiouskarmadog<br>Aj The Epic (2) - LlamaFluff, petroleumjelly<br>LlamaFluff (2) - Aj The Epic, TKoE<br><br><span style="font-style: italic">Not Voting (1) - OceanWind</span><br><br>With 11 votes in play, it will take 6 to lynch.<br><br>The deadline is Sunday, January 17, 2016, at 2:30 AM EST (UTC-5), which is in <span class="countdown"><span style="font-style:italic;">(expired on 2016-01-17 02:30:00)</span></span>.</fieldset></span></span><br><div style="text-align: center"><div alig

['Aj The Epic'] - 2
['LlamaFluff']
['petroleumjelly']

['LlamaFluff'] - 2
['Aj The Epic']
['TKoE']

['TKoE'] - 6
['Raskolnikov']
['Elyse']
['Xtoxm', 'MattP']
['Alchemist21']
['Supreme Overlord']
['curiouskarmadog']

Not Voting - 2
['Marquis']
['OceanWind']



['Aj The Epic'] - 7
['LlamaFluff']
['petroleumjelly']
['Raskolnikov']
['Xtoxm', 'MattP']
['Elyse']
['OceanWind']
['curiouskarmadog']

['LlamaFluff'] - 1
['TKoE']

['Raskolnikov'] - 2
['Alchemist21']
['Aj The Epic']

Not Voting - 2
['Marquis']
['Supreme Overlord']



In [39]:
gameposts[:int(postnumber)]

[{'forum': '53',
  'thread': '62260',
  'pagelink': 'https://forum.mafiascum.net/viewtopic.php?f=53&t=62260&start=0',
  'number': '0',
  'timestamp': 'Thu Jun 18, 2015 12:53 pm',
  'user': 'Ozgin',
  'content': '<div style="text-align: center"><span class="noboldsig"><span style="color: #FF4000"><span class="nocolorsig"><span style="font-size: 250%; line-height: 116%;">Welcome to Mafia Mini 1690 - Zed\'s Tavern!</span></span></span></span><br><img src="http://forums.3dtotal.com/attachment.php?attachmentid=195600&amp;amp;stc=1&amp;amp;d=1344818372" alt="Image"><br><span style="font-style: italic">You walk into the same bar you\'ve frequented for years, Zed\'s Tavern. However, you\'ve noticed lately there have been some shady folks around, doing shady things. You don\'t know them, but you and your buddies are determined to find out who they are. You sit at the bar with 12 others and a bartender. You begin to make a discussion - You are want to beat these troublemakers to a bloody pulp, b